* 估計成功申請貸款機率
<ul>
    <li>Linear Regression</li>
    <li>Logistic Regression</li>
    <li>Decision Tree</li>
    <li>Random Forest</li>
</ul>

In [1]:
import numpy as np
import pandas as pd

In [2]:
acceptdf = pd.read_csv("D:\\Lending Club\\Accepted Data.csv", low_memory=False)
denydf = pd.read_csv("D:\\Lending Club\\Rejected Data.csv", low_memory=False)

In [3]:
acceptdf = acceptdf.drop([2260699, 2260700], axis=0)

In [4]:
acceptdf.head(3)

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,68407277,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,...,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,N,Cash,N
1,68355089,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,...,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,N,Cash,N
2,68341763,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,truck driver,...,50.0,0.0,0.0,218418.0,18696.0,6200.0,14877.0,N,Cash,N


In [5]:
successdf = pd.DataFrame(columns=["Success", "Score", "Amount", "D/I Ratio", "Employment Length", "State"])
denydf = denydf.drop(["Application Date", "Loan Title","Zip Code", "Policy Code"], axis=1)
denydf.columns = ["Amount", "Score", "D/I Ratio", "State", "Employment Length"]

In [6]:
denydf.head(3)

,Amount,Score,D/I Ratio,State,Employment Length
0,1000.0,693.0,10%,NM,4 years
1,1000.0,703.0,10%,MA,< 1 year
2,11000.0,715.0,10%,MD,1 year


In [7]:
def to_string(x):
    if x[2] == "+":
        return 10
    elif x[0] == "<":
        return 0
    else:
        return int(x[0])

col = acceptdf.loc[: , "fico_range_low": "fico_range_high"]
acceptdf["Score"] = col.mean(axis=1)
temptdf = acceptdf[["loan_amnt", "Score", "dti", "addr_state", "emp_length"]]
temptdf.columns = ["Amount", "Score", "D/I Ratio", "State", "Employment Length"]
temptdf = temptdf.dropna(axis=0)
temptdf['Amount'] = temptdf['Amount'].apply(lambda x: int(x))
temptdf['Score'] = temptdf['Score'].apply(lambda x: int(x))
temptdf['Employment Length'] = temptdf['Employment Length'].apply(lambda x: to_string(x))

In [8]:
temptdf["Success"] = 1
Success = temptdf['Success']
temptdf.drop(labels=['Success'], axis=1, inplace = True)
temptdf.insert(0, 'Success', Success)

temptdf.head()

,Success,Amount,Score,D/I Ratio,State,Employment Length
0,1,3600,677,5.91,PA,10
1,1,24700,717,16.06,SD,10
2,1,20000,697,10.78,IL,10
3,1,35000,787,17.06,NJ,10
4,1,10400,697,25.37,PA,3


In [9]:
denydf = denydf.dropna(axis=0)
denydf['Amount'] = denydf['Amount'].apply(lambda x: int(x))
denydf['Score'] = denydf['Score'].apply(lambda x: int(x))
denydf['D/I Ratio'] = denydf['D/I Ratio'].apply(lambda x: float(x[:-1]))
denydf['Employment Length'] = denydf['Employment Length'].apply(lambda x: to_string(x))

In [10]:
denydf = denydf.iloc[:2113648,:]

In [11]:
denydf["Success"] = 0
Success = denydf['Success']
denydf.drop(labels=['Success'], axis=1, inplace = True)
denydf.insert(0, 'Success', Success)
denydf.head()

,Success,Amount,Score,D/I Ratio,State,Employment Length
0,0,1000,693,10.00,NM,4
1,0,1000,703,10.00,MA,0
2,0,11000,715,10.00,MD,1
3,0,6000,698,38.64,MA,0
4,0,1500,509,9.43,MD,0


In [12]:
frames = [successdf, denydf, temptdf]
result = pd.concat(frames, ignore_index=True, sort=False)
result.columns = ["Success", "Score", "Amount", "DIRatio", "EmploymentLength", "State"]

In [13]:
a = result[["Success"]].values.tolist()
b = [i[0] for i in a]
print(b.count(0), b.count(1))

2113648 2113648


In [15]:
result.to_csv("D:\\Lending Club\\Success.csv", index=False)